In [1]:
!pip install tensorflow

In [2]:
import zipfile
import os
import shutil
import random
import gdown
from tqdm.auto import tqdm

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import cv2
from PIL import Image, ImageOps
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [3]:
RANDOM_STATE = 42
random.seed(RANDOM_STATE)

In [4]:
file_id = "1FKZ9oHZ3zFMoFJX2f2aI34M2XZ2ikSb0"
gdown.download(
    f"https://drive.google.com/uc?id={file_id}",
    os.path.join(os.getcwd(), "dataset_32_classes.zip"),
    quiet=False,
)
zip_name = "dataset_32_classes.zip"

Downloading...
From (original): https://drive.google.com/uc?id=1FKZ9oHZ3zFMoFJX2f2aI34M2XZ2ikSb0
From (redirected): https://drive.google.com/uc?id=1FKZ9oHZ3zFMoFJX2f2aI34M2XZ2ikSb0&confirm=t&uuid=bb095551-1019-4005-9df7-1ae7bb5c7a75
To: /content/dataset_32_classes.zip
100%|██████████| 641M/641M [00:06<00:00, 95.9MB/s]


In [5]:
# Распаковка архива
with zipfile.ZipFile(zip_name, "r") as zip_ref:
    zip_ref.extractall("./dataset")

In [6]:
DATASET_DIR = "./dataset"
TEMP_DIR = "./temp"

In [7]:
def set_image_size(img_path, save_path, size, color_background="white"):
    img = Image.open(img_path)
    if img.mode != "RGB":
        img = img.convert("RGB")
    ratio = img.width / img.height
    # Широкое изображение
    if ratio > 1:
        new_width = size[0]
        new_height = int(size[0] / ratio)
    # Высокое изображение
    else:
        new_height = size[1]
        new_width = int(size[1] * ratio)
    img_resized = img.resize((new_width, new_height), Image.LANCZOS)
    img_padded = ImageOps.pad(img_resized, size, color=color_background, centering=(0.5, 0.5))
    img_padded.save(save_path)

In [8]:
def creat_temp_dataset(num_temp=None, use_sample=True):
    if os.path.exists(TEMP_DIR):
        shutil.rmtree(TEMP_DIR)
    os.mkdir(TEMP_DIR)

    for class_name in tqdm(os.listdir(DATASET_DIR)):
        temp_class_path = os.path.join(TEMP_DIR, class_name)
        if os.path.exists(temp_class_path) != True:
            os.mkdir(temp_class_path)
        basedir_class_path = os.path.join(DATASET_DIR, class_name)
        image_names = os.listdir(basedir_class_path)
        if use_sample and num_temp is not None:
            # Если нужно использовать sample, и указано количество
            reduce_image_names = random.sample(image_names, min(num_temp, len(image_names)))
        else:
            # Копируем все изображения
            reduce_image_names = image_names

        for image in reduce_image_names:
            shutil.copy(
                os.path.join(basedir_class_path, image), os.path.join(temp_class_path, image)
            )

In [9]:
def resize_temp_dataset(size, color_background):
    # Проверка, существует ли временный датасет
    if not os.path.exists(TEMP_DIR):
        print("Временный датасет TEMP_DIR не найден.")
        return

    # Проходим по всем классам (папкам) в TEMP_DIR
    for class_name in tqdm(os.listdir(TEMP_DIR)):
        class_path = os.path.join(TEMP_DIR, class_name)

        # Проверяем, является ли это папкой
        if os.path.isdir(class_path):
            for image_name in os.listdir(class_path):
                image_path = os.path.join(class_path, image_name)

                # Проверяем, является ли это изображением
                if image_name.lower().endswith((".png", ".jpg", ".jpeg")):
                    save_path = image_path  # Сохраняем под тем же именем
                    set_image_size(image_path, save_path, size, color_background)

Создаем выборочный датасет из 500 изображений в каждом классе

In [ ]:
creat_temp_dataset(num_temp=500, use_sample=True)

  0%|          | 0/32 [00:00<?, ?it/s]

Приводим все изображения в выборочном датасете к размеру 224x224 px (для использования ResNet50)

In [ ]:
resize_temp_dataset((224, 224), color_background="white")

  0%|          | 0/32 [00:00<?, ?it/s]

In [10]:
# Загрузка модели ResNet
resn50 = ResNet50(weights="imagenet", include_top=False, pooling="avg")

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:
def get_X_and_y(color="color"):
    X = []
    y = []
    for class_name in tqdm(os.listdir(TEMP_DIR)):
        path_class = os.path.join(TEMP_DIR, class_name)

        for img in os.listdir(path_class):
            img_path = os.path.join(path_class, img)

            # Загружаем изображение
            if color == "grey":
                img = load_img(img_path, color_mode="grayscale")  # Загрузка в градациях серого
                img = img_to_array(img)
                img = np.repeat(img, 3, axis=-1)  # Преобразуем в RGB
            else:
                img = load_img(img_path)  # Загрузка цветного изображения
                img = img_to_array(img)

            # Добавляем размерность для модели
            img = np.expand_dims(img, axis=0)
            features = resn50.predict(img)  # Извлекаем признаки

            X.append(features.flatten())  # Плоский массив
            y.append(class_name)

    return np.array(X), np.array(y)

In [ ]:
X, y = get_X_and_y(color="color")

  0%|          | 0/32 [00:00<?, ?it/s]

Выходные данные были обрезаны до нескольких последних строк (5000).
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/s

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {"C": [1, 4, 7, 10], "kernel": ["rbf"], "gamma": ["scale"]}
svс = GridSearchCV(SVC(), param_grid)
svс.fit(X_train, y_train)
y_pred = svс.predict(X_test)

In [ ]:
svс.best_params_, svс.best_score_

({'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}, 0.9857812499999999)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       Apple       0.96      0.96      0.96        94
     Avocado       0.97      0.96      0.97       103
      Banana       0.97      0.93      0.95        90
        Bean       1.00      1.00      1.00       110
Bitter_Gourd       0.99      1.00      1.00       124
Bottle_Gourd       1.00      1.00      1.00       108
     Brinjal       0.99      1.00      1.00       103
    Broccoli       1.00      0.99      0.99       100
     Cabbage       1.00      1.00      1.00        83
    Capsicum       1.00      1.00      1.00        97
      Carrot       1.00      1.00      1.00        98
 Cauliflower       1.00      1.00      1.00        93
      Cherry       0.98      0.97      0.97       117
    Cucumber       1.00      1.00      1.00        97
       Grape       1.00      1.00      1.00       111
        Kiwi       0.99      0.99      0.99        94
       Mango       0.89      0.97      0.93        87
         Nut       1.00    

Попытаемся получше подобрать параметр C, тем самым улучшив метрики

In [ ]:
param_grid = {"C": [9, 11, 12, 13], "kernel": ["rbf"], "gamma": ["scale"]}
svс = GridSearchCV(SVC(), param_grid)
svс.fit(X_train, y_train)
y_pred = svс.predict(X_test)

In [ ]:
svс.best_params_, svс.best_score_

({'C': 9, 'gamma': 'scale', 'kernel': 'rbf'}, 0.9857812499999999)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       Apple       0.96      0.96      0.96        94
     Avocado       0.97      0.96      0.97       103
      Banana       0.97      0.93      0.95        90
        Bean       1.00      1.00      1.00       110
Bitter_Gourd       0.99      1.00      1.00       124
Bottle_Gourd       1.00      1.00      1.00       108
     Brinjal       0.99      1.00      1.00       103
    Broccoli       1.00      0.99      0.99       100
     Cabbage       1.00      1.00      1.00        83
    Capsicum       1.00      1.00      1.00        97
      Carrot       1.00      1.00      1.00        98
 Cauliflower       1.00      1.00      1.00        93
      Cherry       0.98      0.97      0.97       117
    Cucumber       1.00      1.00      1.00        97
       Grape       1.00      1.00      1.00       111
        Kiwi       0.99      0.99      0.99        94
       Mango       0.89      0.97      0.93        87
         Nut       1.00    

In [ ]:
param_grid = {"C": np.arange(8.5, 10.1, 0.1), "kernel": ["rbf"], "gamma": ["scale"]}
svс = GridSearchCV(SVC(), param_grid)
svс.fit(X_train, y_train)
y_pred = svс.predict(X_test)

In [ ]:
svс.best_params_, svс.best_score_

({'C': 8.5, 'gamma': 'scale', 'kernel': 'rbf'}, 0.9857812499999999)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       Apple       0.96      0.96      0.96        94
     Avocado       0.97      0.96      0.97       103
      Banana       0.97      0.93      0.95        90
        Bean       1.00      1.00      1.00       110
Bitter_Gourd       0.99      1.00      1.00       124
Bottle_Gourd       1.00      1.00      1.00       108
     Brinjal       0.99      1.00      1.00       103
    Broccoli       1.00      0.99      0.99       100
     Cabbage       1.00      1.00      1.00        83
    Capsicum       1.00      1.00      1.00        97
      Carrot       1.00      1.00      1.00        98
 Cauliflower       1.00      1.00      1.00        93
      Cherry       0.98      0.97      0.97       117
    Cucumber       1.00      1.00      1.00        97
       Grape       1.00      1.00      1.00       111
        Kiwi       0.99      0.99      0.99        94
       Mango       0.89      0.97      0.93        87
         Nut       1.00    

In [ ]:
X_grey, y_grey = get_X_and_y("grey")

  0%|          | 0/32 [00:00<?, ?it/s]

Выходные данные были обрезаны до нескольких последних строк (5000).
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/s

In [ ]:
X_grey_train, X_grey_test, y_grey_train, y_grey_test = train_test_split(
    X_grey, y_grey, test_size=0.2, random_state=42
)

In [ ]:
param_grid = {"C": [1, 4, 7, 10, 12], "kernel": ["rbf"], "gamma": ["scale"]}
svс_grey = GridSearchCV(SVC(), param_grid)
svс_grey.fit(X_train, y_train)
y_grey_pred = svс_grey.predict(X_grey_test)

In [ ]:
svс_grey.best_params_, svс_grey.best_score_

({'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}, 0.9857812499999999)

In [ ]:
print(classification_report(y_grey_test, y_grey_pred))

              precision    recall  f1-score   support

       Apple       0.51      0.62      0.56        94
     Avocado       0.39      0.92      0.55       103
      Banana       0.72      0.91      0.80        90
        Bean       0.89      1.00      0.94       110
Bitter_Gourd       0.98      0.98      0.98       124
Bottle_Gourd       1.00      0.97      0.99       108
     Brinjal       0.91      0.84      0.87       103
    Broccoli       0.99      0.99      0.99       100
     Cabbage       1.00      0.99      0.99        83
    Capsicum       0.99      0.88      0.93        97
      Carrot       0.99      0.94      0.96        98
 Cauliflower       1.00      0.96      0.98        93
      Cherry       0.86      0.79      0.83       117
    Cucumber       0.98      0.81      0.89        97
       Grape       0.85      0.95      0.90       111
        Kiwi       0.44      0.64      0.52        94
       Mango       1.00      0.40      0.57        87
         Nut       0.93    

Обучим на полном датасете цветных фотографий

In [12]:
creat_temp_dataset(use_sample=False)

  0%|          | 0/32 [00:00<?, ?it/s]

In [13]:
resize_temp_dataset((224, 224), color_background="white")

  0%|          | 0/32 [00:00<?, ?it/s]

In [14]:
X_full, y_full = get_X_and_y(color="color")

  0%|          | 0/32 [00:00<?, ?it/s]

Выходные данные были обрезаны до нескольких последних строк (5000).
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/s

In [15]:
X_full_train, X_full_test, y_full_train, y_full_test = train_test_split(
    X_full, y_full, test_size=0.2, random_state=42
)

svc_full = SVC(C=8.5, kernel="rbf")
svc_full.fit(X_full_train, y_full_train)
y_full_pred = svc_full.predict(X_full_test)

In [16]:
y_pred_train = svc_full.predict(X_full_train)
accuracy_score(y_full_train, y_pred_train)

1.0

In [20]:
print(classification_report(y_full_test, y_full_pred))

              precision    recall  f1-score   support

       Apple       0.99      0.96      0.97       295
     Avocado       0.97      0.99      0.98       271
      Banana       0.99      0.98      0.98       302
        Bean       1.00      1.00      1.00       303
Bitter_Gourd       1.00      1.00      1.00       294
Bottle_Gourd       1.00      1.00      1.00       259
     Brinjal       1.00      1.00      1.00       296
    Broccoli       1.00      1.00      1.00       259
     Cabbage       1.00      1.00      1.00       296
    Capsicum       1.00      1.00      1.00       289
      Carrot       1.00      1.00      1.00       261
 Cauliflower       1.00      1.00      1.00       297
      Cherry       0.97      0.98      0.98       266
    Cucumber       1.00      1.00      1.00       266
       Grape       1.00      1.00      1.00       287
        Kiwi       0.98      0.99      0.99       270
       Mango       0.95      0.96      0.96       253
         Nut       1.00    

In [23]:
accuracy_score(y_full_test, y_full_pred)

0.9933035714285714

Для нахождения лучших гиперпараметров использовался RandomizedSearchCV. Для оценки производительности модели применялась метрика accuracy, так как даёт хорошие результаты на сбалансированных классах.**


Результаты обучения с помощью ResNet50 + SVM для выборки из 500 изображений каждого класса

|Модель|Гиперпараметры|Размер изображения|Цветное|accuracy на трейне|accuracy на test|
|:----:|:----:|:----:|:----:|:----:|:----:|
|SVM|C=10, kernel='rbf'|224px|да|0.99|0.99|
|SVM|C=9, kernel='rbf'|224px|да|0.99|0.99|
|SVM|C=8.5, kernel='rbf'|224px|да|0.99|0.99|
|SVM|C=10, kernel='rbf'|224px|нет|0.99|0.82|

Выводы:
- Как мы видим, при разных значениях гиперпараметра С модель выдает одинаковые результаты метрик. Это может быть связано с тем,что признаки, извлеченные с помощью ResNet50, могут быть очень информативными и хорошо разделяющими классы, что делает модель SVM менее чувствительной к параметрам C. Если данные хорошо разделены, модель будет давать схожие результаты при различных значениях параметров. Поэтому для обучения на полном датасете был взят гиперпараметр С = 8.5(лучший по мнению GridSearchCV).
- Модель, обученная на черно-белых изображениях, показала accuracy на трейне, сравнимое со значением модели, обученной на цветных изображениях, но на test-выборке показатели сильно упали, что может говорить о переобучении. Поэтому модель будет обучаться на полном датасете с цветными фотографиями.

**Результаты обучения c помощью ResNet50 + SVM на полном датасете:**

|Модель|Гиперпараметры|Размер изображения|Цветное|accuracy на трейне|accuracy на test|
|:----:|:----:|:----:|:----:|:----:|:----:|
|SVM|C=8.5, kernel='rbf'|224px|да|1.0|0.99|

**Итог: модель, обученная на полном датасете цветных фотографий, показала значение метрики accuracy на test-выборке близкое к 1, что говорит о хорошем обучении модели.**